In [ ]:
# Install required packages
!pip uninstall nltk -y -q
!pip install pandas numpy nltk Sastrawi wordcloud seaborn matplotlib scikit-learn -q

import pandas as pd
import numpy as np
import re
import logging
from nltk.tokenize import word_tokenize
import nltk
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Clear NLTK data and reinstall
import shutil
import os
for path in nltk.data.path:
    if os.path.exists(path):
        shutil.rmtree(path, ignore_errors=True)

nltk.download('punkt')
nltk.download('punkt_tab')

# Inisialisasi logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Load data preprocessed
try:
    df = pd.read_csv('ikn_news_preprocessed_final.csv')  # Perbaikan: Ganti file dan tambah tanda kutip penutup
    logger.info("Berhasil memuat file ikn_news_preprocessed_final.csv")
except FileNotFoundError:
    logger.error("ikn_news_preprocessed_final.csv tidak ditemukan.")
    raise

# Pastikan kolom yang diperlukan ada
required_columns = ['filtered_content', 'label']  # Gunakan filtered_content yang sudah dipreproses
if not all(col in df.columns for col in required_columns):
    missing_cols = [col for col in required_columns if col not in df.columns]
    logger.error(f"Kolom yang hilang: {missing_cols}")
    raise ValueError(f"DataFrame tidak memiliki kolom yang diperlukan: {missing_cols}")

# Pisahkan fitur (teks) dan label
X = df['filtered_content'].fillna('')  # Gunakan filtered_content sebagai input
y = df['label']

# Bagi data menjadi train dan test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Definisikan pipeline dengan TF-IDF dan MultinomialNB
pipeline_nb = make_pipeline(
    TfidfVectorizer(
        stop_words=['yang', 'dan', 'di', 'dengan', 'untuk', 'pada', 'ini', 'dari', 'ke', 'dalam', 'tersebut', 'adalah', 'oleh'],
        max_features=5000,
        ngram_range=(1, 2)  # Tambahkan bigram untuk konteks lebih baik
    ),
    MultinomialNB()
)

# Definisikan parameter grid untuk tuning
param_grid = {
    'tfidfvectorizer__max_features': [3000, 5000, 7000],
    'multinomialnb__alpha': [0.1, 0.5, 1.0]
}

# Lakukan GridSearchCV untuk menemukan parameter terbaik
grid_search = GridSearchCV(pipeline_nb, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Cetak parameter terbaik
print(f"Parameter terbaik: {grid_search.best_params_}")
print(f"Skor terbaik: {grid_search.best_score_:.2f}")

# Evaluasi model terbaik
y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAkurasi Model pada Test Set: {accuracy:.2f}")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred))

# Prediksi teks baru
new_texts = [
    "Proyek IKN berkembang pesat dengan dukungan pemerintah",
    "Kontroversi kebijakan IKN memicu debat publik"
]
predictions = grid_search.predict(new_texts)

# Menampilkan hasil prediksi
for text, pred in zip(new_texts, predictions):
    print(f"\nTeks: '{text}'")
    print(f"Kategori Prediksi: {pred}")

# Visualisasi distribusi label
plt.figure(figsize=(8, 6))
sns.countplot(x='label', data=df, palette='viridis')
plt.title('Distribusi Label', fontsize=16)
plt.xlabel('Label', fontsize=14)
plt.ylabel('Jumlah Data', fontsize=14)
plt.tight_layout()
plt.savefig('label_distribution.png')
plt.show()

In [ ]:
# Buka terminal (Ctrl + `) dan jalankan:
pip install pandas numpy scikit-learn seaborn matplotlib joblib

In [ ]:
# In [ ]:
import pandas as pd
import numpy as np
import logging
import seaborn as sns
import matplotlib.pyplot as plt
import os
import joblib

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Inisialisasi logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Buat folder yang diperlukan jika belum ada
os.makedirs('data', exist_ok=True)
os.makedirs('images', exist_ok=True)
os.makedirs('models', exist_ok=True)

# Load data hasil preprocessing akhir (dari notebook 04)
try:
    df = pd.read_csv('data/gojek_news_preprocessed_final.csv')
    logger.info("Berhasil memuat file 'gojek_news_preprocessed_final.csv'")
except FileNotFoundError:
    logger.error("'gojek_news_preprocessed_final.csv' tidak ditemukan. Pastikan notebook 04 sudah dijalankan.")
    raise

In [ ]:
# In [ ]:
# Pastikan kolom yang diperlukan ada
required_columns = ['filtered_content', 'label']
if not all(col in df.columns for col in required_columns):
    raise ValueError(f"DataFrame tidak memiliki kolom yang diperlukan: {required_columns}")

# Pisahkan fitur (X) dan label (y)
X = df['filtered_content'].fillna('')  # Mengisi nilai NaN dengan string kosong
y = df['label']

# Bagi data menjadi data latih (train) dan data uji (test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

logger.info(f"Ukuran data latih: {len(X_train)} sampel")
logger.info(f"Ukuran data uji: {len(X_test)} sampel")

# Definisikan pipeline model
pipeline_nb = make_pipeline(
    TfidfVectorizer(
        ngram_range=(1, 2)  # Menggunakan unigram dan bigram
    ),
    MultinomialNB()
)

In [ ]:
# In [ ]:
# Definisikan parameter grid untuk diuji
param_grid = {
    'tfidfvectorizer__max_features': [3000, 5000, 7000],
    'tfidfvectorizer__max_df': [0.75, 0.9, 1.0],
    'multinomialnb__alpha': [0.1, 0.5, 1.0]
}

# Lakukan pencarian parameter terbaik menggunakan cross-validation
logger.info("Memulai GridSearchCV untuk mencari parameter terbaik...")
grid_search = GridSearchCV(pipeline_nb, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Tampilkan hasil terbaik
print("\n--- Hasil GridSearchCV ---")
print(f"Parameter terbaik: {grid_search.best_params_}")
print(f"Skor akurasi cross-validation terbaik: {grid_search.best_score_:.3f}")

In [ ]:
# In [ ]:
# Gunakan model terbaik untuk prediksi pada data uji
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=3)

print(f"\nAkurasi Model pada Data Uji: {accuracy:.3f}")
print("\nLaporan Klasifikasi:\n")
print(report)

# Visualisasi Confusion Matrix
cm = confusion_matrix(y_test, y_pred, labels=best_model.classes_)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=best_model.classes_, yticklabels=best_model.classes_)
plt.title('Confusion Matrix - TF-IDF + MultinomialNB (Gojek)', fontsize=15)
plt.xlabel('Prediksi', fontsize=12)
plt.ylabel('Aktual', fontsize=12)
plt.savefig('images/gojek_tfidf_confusion_matrix.png')
plt.show()

In [ ]:
# In [ ]:
# Contoh kalimat baru yang relevan dengan Gojek
new_texts = [
    "Layanan gojek sangat membantu mobilitas saya sehari-hari, cepat dan efisien.",
    "Tarif ojol sekarang terasa terlalu mahal dan sering kali orderan dibatalkan oleh driver.",
    "Gojek mengumumkan akan memperluas jangkauan operasional ke beberapa kota baru."
]

predictions = best_model.predict(new_texts)
prediction_proba = best_model.predict_proba(new_texts)

print("--- Hasil Prediksi pada Kalimat Baru ---\n")
for text, pred, proba in zip(new_texts, predictions, prediction_proba):
    print(f"Teks: '{text}'")
    print(f"Prediksi Sentimen: {pred}")
    # Tampilkan probabilitas untuk setiap kelas
    proba_dict = {label: f"{p:.2%}" for label, p in zip(best_model.classes_, proba)}
    print(f"Probabilitas: {proba_dict}\n")

In [ ]:
# In [ ]:
model_path = 'models/tfidf_nb_model.joblib'
joblib.dump(best_model, model_path)
logger.info(f"Model terbaik telah disimpan di: {model_path}")

# Contoh cara memuat model kembali:
# loaded_model = joblib.load(model_path)
# print("Model berhasil dimuat kembali.")